In [13]:
import glob

import numpy as np
import pandas as pd
from scipy import signal

import librosa
import librosa.display as DSP
import librosa as lr
from tempfile import mktemp
import imagehash
from PIL import Image

import pylab

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

import os
import wave
import time
import pickle
import warnings
import numpy as np
from sklearn import preprocessing
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
from sklearn.metrics import f1_score
import joblib

In [14]:
def calculate_delta(array):
   
    rows, cols = array.shape
    # print(rows)
    # print(cols)
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first =0
            else:
                first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

In [15]:
def extract_features(audio,rate):
       
    mfcc_feature = mfcc.mfcc(audio, rate, 0.025, 0.01, 20, nfft = 2205, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
#     print(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature, delta)) 
    return combined

In [16]:
X_test = []
y_test = []

lstFolders = ['Doha', 'shuaib', 'rabea', 'Rahma']
for lst in lstFolders:
    listFiles = glob.glob(r"Training Data\{}\*".format(lst))
    features = np.asarray(())
    finishCounter = len(listFiles)
    counter = 1
    for file in listFiles:
        try:
            audio, sr = librosa.load(file)
            vector = extract_features(audio, sr)
            
            if features.size == 0:
                features = vector
            else:
                features = np.vstack((features, vector))
                
            X_test.append(vector)
            y_test.append(lstFolders.index(lst))
                
            counter += 1
            if(counter == finishCounter):
                gmm = GaussianMixture(n_components = 6, max_iter = 200, covariance_type='diag', n_init = 3)
                gmm.fit(features)
                
                # dumping the trained gaussian model
                picklefile = lst + ".joblib"
                joblib.dump(gmm, picklefile) 
        except:
            continue

In [17]:
gmm_files = [ i + '.joblib' for i in ['Doha', 'shuaib', 'rabea', 'Rahma']]

models    = [joblib.load(fname) for fname in gmm_files]

In [18]:
final_y = []
y = []
for i in X_test:
    log_likelihood = np.zeros(len(models)) 
    for j in range(len(models)):
        gmm = models[j] 
        scores = np.array(gmm.score(i))
        log_likelihood[j] = scores.sum()
    y.append(log_likelihood)

    winner = np.argmax(log_likelihood)
    final_y.append(winner)

In [19]:
f1_score(final_y, y_test, average='micro')

1.0

In [20]:
file = r'C:\Users\DELL\Desktop\simple-recorder3\Training Data word\Open_book\rahma_open_book6 (4).wav'
audio, sr = librosa.load(file)
vector = extract_features(audio, sr)
og_likelihood = np.zeros(len(models)) 
for i in range(len(models)):
    gmm    = models[i] 
    scores = np.array(gmm.score(vector))
    log_likelihood[i] = scores.sum()
y.append(log_likelihood)

winner = np.argmax(log_likelihood)
print("\tdetected as -",models[winner])
print(winner)

	detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
3


In [22]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2) 
X_principal = pca.fit_transform(audio) 
X_principal = pd.DataFrame(X_principal) 
X_principal.columns = ['P1', 'P2'] 
import matplotlib.pyplot as plt 
plt.scatter(X_principal[1], X_principal[2],
           c = GaussianMixture(n_components = 2).fit_predict(X_principal), cmap =plt.cm.winter, alpha = 0.6) 
plt.show() 

ValueError: Expected 2D array, got 1D array instead:
array=[0.         0.         0.         ... 0.00222425 0.0028127  0.        ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.